# Advanced Analysis: User Segmentation

## Why Segmentation Matters

Overall A/B test results can hide important insights. What if the new button works great for mobile users but poorly for desktop? Or what if it performs differently by age group or location?

**This notebook demonstrates:**
- How to segment users by demographics and behavior
- Running A/B tests within each segment
- Identifying which user groups benefit most from changes
- Making targeted rollout decisions

---

## 1. Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 2. Generate Segmented Data

We'll create realistic user data with multiple segments:
- **Device:** Mobile vs Desktop
- **User Type:** New vs Returning
- **Age Group:** 18-34, 35-54, 55+

**Key Insight:** The new button might perform differently across segments!

In [ ]:
def generate_segmented_data(n_total=10000):
    """
    Generate A/B test data with user segments.
    
    Key assumptions:
    - Mobile users respond better to the new button (+3% lift)
    - Desktop users see smaller improvement (+1.5% lift)
    - New users benefit more than returning users
    """
    
    # Equal split between control and treatment
    n_per_group = n_total // 2
    
    data = []
    user_id = 1
    
    for group in ['Control', 'Treatment']:
        for i in range(n_per_group):
            # Randomly assign device (60% mobile, 40% desktop)
            device = np.random.choice(['Mobile', 'Desktop'], p=[0.6, 0.4])
            
            # Randomly assign user type (70% returning, 30% new)
            user_type = np.random.choice(['Returning', 'New'], p=[0.7, 0.3])
            
            # Randomly assign age group
            age_group = np.random.choice(['18-34', '35-54', '55+'], p=[0.4, 0.35, 0.25])
            
            # Base conversion rates by segment
            if device == 'Mobile':
                base_rate = 0.13 if user_type == 'New' else 0.11
            else:
                base_rate = 0.12 if user_type == 'New' else 0.10
            
            # Treatment effect varies by segment
            if group == 'Treatment':
                if device == 'Mobile' and user_type == 'New':
                    conversion_rate = base_rate + 0.035  # +3.5% for mobile new users
                elif device == 'Mobile':
                    conversion_rate = base_rate + 0.025  # +2.5% for mobile returning
                elif user_type == 'New':
                    conversion_rate = base_rate + 0.020  # +2.0% for desktop new
                else:
                    conversion_rate = base_rate + 0.010  # +1.0% for desktop returning
            else:
                conversion_rate = base_rate
            
            # Generate conversion
            converted = np.random.binomial(1, conversion_rate)
            
            data.append({
                'user_id': user_id,
                'group': group,
                'device': device,
                'user_type': user_type,
                'age_group': age_group,
                'converted': converted
            })
            
            user_id += 1
    
    return pd.DataFrame(data)

# Generate data
df = generate_segmented_data(10000)

print("Dataset Preview:")
print(df.head(10))
print(f"\nTotal Users: {len(df):,}")
print(f"\nSegment Distribution:")
print(df[['device', 'user_type', 'age_group']].describe())

## 3. Overall Results (Before Segmentation)

In [ ]:
# Overall conversion rates
overall = df.groupby('group')['converted'].agg(['sum', 'count', 'mean'])
overall.columns = ['Conversions', 'Users', 'Rate']
overall['Rate_Pct'] = (overall['Rate'] * 100).round(2)

print("Overall A/B Test Results:")
print(overall)

control_rate = overall.loc['Control', 'Rate']
treatment_rate = overall.loc['Treatment', 'Rate']
overall_lift = treatment_rate - control_rate

print(f"\nOverall Lift: {overall_lift:.2%}")
print(f"Relative Improvement: {(treatment_rate/control_rate - 1)*100:.1f}%")

## 4. Segmentation Analysis by Device

In [ ]:
def analyze_segment(df, segment_col, segment_name):
    """
    Analyze A/B test results for a specific segment.
    """
    results = []
    
    for segment_value in df[segment_col].unique():
        segment_data = df[df[segment_col] == segment_value]
        
        control = segment_data[segment_data['group'] == 'Control']['converted']
        treatment = segment_data[segment_data['group'] == 'Treatment']['converted']
        
        control_rate = control.mean()
        treatment_rate = treatment.mean()
        lift = treatment_rate - control_rate
        
        # Statistical test
        t_stat, p_value = stats.ttest_ind(treatment, control)
        
        results.append({
            segment_name: segment_value,
            'Control_Rate': control_rate,
            'Treatment_Rate': treatment_rate,
            'Absolute_Lift': lift,
            'Relative_Lift_%': (treatment_rate/control_rate - 1) * 100,
            'P_Value': p_value,
            'Significant': 'Yes' if p_value < 0.05 else 'No',
            'Control_N': len(control),
            'Treatment_N': len(treatment)
        })
    
    return pd.DataFrame(results)

# Analyze by device
device_results = analyze_segment(df, 'device', 'Device')

print("="*80)
print("SEGMENTATION BY DEVICE")
print("="*80)
print(device_results.to_string(index=False))
print("\n💡 Key Insight:")
if device_results['Absolute_Lift'].max() > overall_lift * 1.5:
    best_device = device_results.loc[device_results['Absolute_Lift'].idxmax(), 'Device']
    print(f"The new button performs significantly better on {best_device} devices!")
    print(f"Consider a device-targeted rollout strategy.")

In [ ]:
# Visualize device segmentation
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart by device
ax1 = axes[0]
device_pivot = df.groupby(['device', 'group'])['converted'].mean().unstack() * 100
device_pivot.plot(kind='bar', ax=ax1, color=['#3498db', '#e74c3c'], alpha=0.8, edgecolor='black')
ax1.set_ylabel('Conversion Rate (%)', fontsize=12, fontweight='bold')
ax1.set_xlabel('Device Type', fontsize=12, fontweight='bold')
ax1.set_title('Conversion Rate by Device & Group', fontsize=14, fontweight='bold')
ax1.legend(['Control', 'Treatment'], title='Group')
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=0)
ax1.grid(axis='y', alpha=0.3)

# Lift by device
ax2 = axes[1]
bars = ax2.bar(device_results['Device'], device_results['Absolute_Lift'] * 100, 
               color=['#2ecc71', '#f39c12'], alpha=0.8, edgecolor='black')
ax2.set_ylabel('Absolute Lift (%)', fontsize=12, fontweight='bold')
ax2.set_xlabel('Device Type', fontsize=12, fontweight='bold')
ax2.set_title('Treatment Lift by Device', fontsize=14, fontweight='bold')
ax2.axhline(y=0, color='red', linestyle='--', linewidth=2, alpha=0.5)
ax2.grid(axis='y', alpha=0.3)

# Add value labels
for bar in bars:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.2f}%', ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

## 5. Segmentation by User Type

In [ ]:
# Analyze by user type
user_type_results = analyze_segment(df, 'user_type', 'User_Type')

print("="*80)
print("SEGMENTATION BY USER TYPE")
print("="*80)
print(user_type_results.to_string(index=False))
print("\n💡 Key Insight:")
if user_type_results['Absolute_Lift'].max() > overall_lift * 1.3:
    best_type = user_type_results.loc[user_type_results['Absolute_Lift'].idxmax(), 'User_Type']
    print(f"{best_type} users respond much better to the new button!")

## 6. Cross-Segmentation (Device × User Type)

In [ ]:
# Create cross-segment
df['segment'] = df['device'] + ' - ' + df['user_type']

# Analyze cross-segments
cross_results = analyze_segment(df, 'segment', 'Segment')
cross_results = cross_results.sort_values('Absolute_Lift', ascending=False)

print("="*80)
print("CROSS-SEGMENTATION: DEVICE × USER TYPE")
print("="*80)
print(cross_results.to_string(index=False))

print("\n🎯 Strategic Recommendation:")
top_segment = cross_results.iloc[0]
worst_segment = cross_results.iloc[-1]

print(f"\nBest performing segment: {top_segment['Segment']}")
print(f"  → Lift: {top_segment['Absolute_Lift']:.2%} ({top_segment['Relative_Lift_%']:.1f}% relative)")
print(f"  → Significant: {top_segment['Significant']}")

print(f"\nWorst performing segment: {worst_segment['Segment']}")
print(f"  → Lift: {worst_segment['Absolute_Lift']:.2%} ({worst_segment['Relative_Lift_%']:.1f}% relative)")
print(f"  → Significant: {worst_segment['Significant']}")

In [ ]:
# Heatmap of cross-segment performance
plt.figure(figsize=(10, 6))

# Create pivot table for heatmap
heatmap_data = df.groupby(['device', 'user_type', 'group'])['converted'].mean().unstack()
heatmap_data['Lift'] = (heatmap_data['Treatment'] - heatmap_data['Control']) * 100
lift_pivot = heatmap_data['Lift'].unstack()

sns.heatmap(lift_pivot, annot=True, fmt='.2f', cmap='RdYlGn', center=0, 
            cbar_kws={'label': 'Lift (%)'},
            linewidths=2, linecolor='white')
plt.title('Treatment Lift by Device & User Type', fontsize=15, fontweight='bold', pad=15)
plt.xlabel('User Type', fontsize=12, fontweight='bold')
plt.ylabel('Device', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

## 7. Business Recommendations by Segment

Based on the segmentation analysis, here are data-driven rollout strategies:

In [ ]:
print("="*80)
print(" " * 25 + "ROLLOUT STRATEGY")
print("="*80)

# Identify segments with significant positive lift
winners = cross_results[
    (cross_results['Significant'] == 'Yes') & 
    (cross_results['Absolute_Lift'] > 0)
]

print("\n✅ RECOMMENDED ROLLOUT SEGMENTS:")
print("-" * 80)
for idx, row in winners.iterrows():
    segment_pct = (row['Control_N'] + row['Treatment_N']) / len(df) * 100
    print(f"\n{row['Segment']}")
    print(f"  • Lift: {row['Absolute_Lift']:.2%} (p-value: {row['P_Value']:.4f})")
    print(f"  • % of user base: {segment_pct:.1f}%")
    print(f"  • Recommendation: IMMEDIATE ROLLOUT")

# Segments to avoid or monitor
losers = cross_results[
    (cross_results['Absolute_Lift'] <= 0) |
    ((cross_results['Significant'] == 'No') & (cross_results['Absolute_Lift'] < overall_lift * 0.5))
]

if len(losers) > 0:
    print("\n\n⚠️  SEGMENTS TO EXCLUDE OR MONITOR:")
    print("-" * 80)
    for idx, row in losers.iterrows():
        segment_pct = (row['Control_N'] + row['Treatment_N']) / len(df) * 100
        print(f"\n{row['Segment']}")
        print(f"  • Lift: {row['Absolute_Lift']:.2%} (p-value: {row['P_Value']:.4f})")
        print(f"  • % of user base: {segment_pct:.1f}%")
        print(f"  • Recommendation: KEEP CURRENT DESIGN or EXTEND TEST")

print("\n" + "="*80)
print("\n📊 FINAL STRATEGY:")
print("\nOption 1: TARGETED ROLLOUT")
print("  → Launch new button only to high-performing segments")
print("  → Monitor performance and expand gradually")

print("\nOption 2: UNIVERSAL ROLLOUT with SEGMENT MONITORING")
print("  → Launch to all users but track segment performance")
print("  → Be ready to roll back for underperforming segments")

print("\nOption 3: SEGMENT-SPECIFIC DESIGNS")
print("  → Create different button variants for different segments")
print("  → Example: New design for Mobile, keep current for Desktop")
print("\n" + "="*80)

## 8. Key Takeaways

### Why Segmentation Analysis Matters:

1. **Reveals Hidden Patterns:** Overall results can mask important segment differences
2. **Enables Targeted Decisions:** Roll out to segments where it works, not everywhere
3. **Reduces Risk:** Avoid harming conversion in segments where treatment underperforms
4. **Maximizes ROI:** Focus resources on highest-impact user groups

### Real-World Applications:

- **Netflix:** Different UI for different device types
- **Amazon:** Personalized checkout experiences by user history
- **Uber:** Surge pricing varies by user segment and location

### Interview Talking Points:

> "I didn't just run an overall A/B test - I segmented users by device and behavior to identify which groups benefited most. This revealed that mobile users had 3x higher lift than desktop users, suggesting a targeted rollout strategy would maximize impact while minimizing risk."

---

*This analysis demonstrates advanced statistical thinking and business acumen - exactly what companies like Disney look for in decision scientists.*